# Interact with `SampleGen` simulation

requires litex_server running and connected to the VC707 USB-uart port:

```bash
# both run at the same time in separate windows ...
$ python3 sim_sample_gen.py
$ litex_server --uart --uart-port socket://localhost:1111 --bind-port 2234
```

In [1]:
import numpy as np
from time import sleep
from litex import RemoteClient
import sys
sys.path.append('../spi')
from client_tools import getId, hd, big_write, setSamples, set_trigger_freq

In [2]:
# Connect to litex server
r = RemoteClient(csr_csv='out/csr.csv', debug=False, port=2234)
r.open()
getId(r)

'LiteX Simulation 2021-04-09 14:02:06'

In [3]:
set_trigger_freq(r, 10e6)

In [4]:
# samples = np.arange(0x80) - 0x40
# samples = np.arange(0x2000) - 0x1000
samples = np.arange(64)
setSamples(r, samples)

In [5]:
assert r.regs.sample_gen_wfm_len.read() == len(samples)//16 -1

In [7]:
# start vcd dump for a few ms, then stop sim.
r.regs.sim_trace_enable.write(1)
sleep(0.0001)
r.regs.sim_trace_enable.write(0)
# r.regs.sim_finish_finish.write(1)

to inspect result, run

```bash
gtkwave out/gateware/sim.vcd
```

look for `TOP/sim/source_converter0` which is the sample stream for the DAC.